In [10]:
import pandas as pd
import numpy as np
from PIL import Image
from sklearn.cluster import KMeans
from tqdm import tqdm
import plotly.express as px

In [24]:
class ClusterImage:
    def __init__(self, image_obj):
        self.image_obj = image_obj
        self.shape = np.shape(np.array(self.image_obj))
        self.image_array = np.reshape(np.array(self.image_obj),(self.shape[0] * self.shape[1], self.shape[2]) )
        self.clustered_image_array = []
        self.cluster_image_obj = None

    def get_clustered_image(self, clusters = 10, clustering_type = "KMeans"):
        match clustering_type:
            case "Kmeans":
                kmeans = KMeans(n_clusters=clusters, random_state=0, max_iter = 10, n_init=1).fit(self.image_array)
                for i, pixel in enumerate(tqdm(self.image_array)):
                    cluster = kmeans.labels_[i]
                    self.clustered_image_array.append([int(samp) for samp in kmeans.cluster_centers_[cluster] ])
        
        self.cluster_image_obj = Image.fromarray(np.reshape(self.clustered_image_array, self.shape))

In [25]:
immy = Image.open("frog.jpg")
imageClustered = ClusterImage(immy)

immy.save("saved.png")
print (imageClustered.image_array)

imageClustered.get_clustered_image()
imageClustered.cluster_image_obj.save("savedCluster.png")

[[  3   3   3]
 [  3   3   3]
 [  3   3   3]
 ...
 [228 228 226]
 [228 228 226]
 [228 228 226]]


ValueError: cannot reshape array of size 0 into shape (1691,2484,3)